In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [61]:
import os
os.chdir("/content/gdrive/My Drive/功率曲线建模/CCFD-HU/probabilistic power curve/new_dataset")

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import math
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import tensorflow_probability as tfp
from keras.models import Sequential
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import StringLookup
import joblib
from tqdm import tqdm

#ignore warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
df_res = pd.read_csv('df_res.csv')

In [5]:
wind_array = df_res.to_numpy()

# 步长为m
m = 500

nrm = [[] for j in range(len(wind_array) // m - 1 )]

for i in range(0, len(wind_array) // m - 1):
    nrm[i] = wind_array[i*(m):i*(m) + m]

QPC

In [6]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform,RandomNormal
from matplotlib import pyplot as plt

tfd = tfp.distributions

In [7]:
def qloss(y_true, y_pred, n_q=99):
    q = np.array(range(1, n_q + 1))
    left = (q / (n_q + 1) - 1) * (y_true - y_pred)
    right = q / (n_q + 1) * (y_true - y_pred)
    return K.mean(K.maximum(left, right))

from keras.callbacks import *

def QRNN(num_quantiles):
  mean_inputs = tf.keras.Input(16)
  mean_x = Dense(256,)(mean_inputs)
  mean_x=Dense(256,)(mean_x)
  mean_x = Dense(128,)(mean_x)
  mean_x = Dense(64,)(mean_x)
  mean_x = Dense(32,)(mean_x)
  outputs=Dense(num_quantiles)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

def QRNN_2(num_quantiles):
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(64, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  outputs=Dense(num_quantiles)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

In [ ]:
# For QiPao Plot
from tqdm import tqdm

# 前20个用的QRNN_2的结构，后面的用的QRNN_3的结构
distance_matrix = np.zeros((500, 14))

i = 368
nrm0 = nrm[i]
speed0 = nrm0[:, 0::2].flatten()

power_list = []

speed_list = np.linspace(np.percentile(speed0, 0.5), np.percentile(speed0, 99.5), 20)
speed_list = speed_list.reshape(len(speed_list), 1)

for j in range(14):
  num_quantiles = 9
  model = QRNN_2(num_quantiles)
  model.load_weights("models4plot/quantile_%s.h5"%j)
  power_list.append(model.predict(speed_list))

power_list = np.array(power_list)
power_list[power_list>1.1] = 1.1
power_list[power_list<0] = 0

for k in range(14):
  tmp = np.delete(power_list, (k), axis = 0)
  distance_all = [(np.mean((tmp[i] - power_list[k])**2))**0.5 for i in range(len(tmp))]
  sort_array = np.argsort(distance_all)[:-3]
  filter_distance = [distance_all[i] for i in sort_array]
  distance_matrix[i][k] = np.mean(filter_distance)

In [ ]:
distance_matrix[i]

array([0.02246418, 0.07281165, 0.02257778, 0.02055283, 0.02263397,
       0.02690489, 0.07162813, 0.02759623, 0.02265561, 0.04322292,
       0.03252698, 0.02271159, 0.02239755, 0.04226735])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)

In [ ]:
# Distance result for quantile power curve
color_array = np.array([0.02246418, 0.07281165, 0.02257778, 0.02055283, 0.02263397,
       0.02690489, 0.07162813, 0.02759623, 0.02265561, 0.04322292,
       0.03252698, 0.02271159, 0.02239755, 0.04226735])

size_array = color_array*1500

pyplt = py.offline.plot

trace0 = go.Scatter(
 x=[10, 20, 30, 40, 50, 10, 20, 30, 40, 50, 10, 20, 30, 40,],
 y=[50, 50, 50, 50, 50, 30, 30, 30, 30, 30, 10, 10, 10, 10,],
 mode='markers',
 marker=dict(
     color=color_array,
     size=size_array,
     showscale= True,
 )
)

layout = go.Layout(
    template = "plotly_white",
)

data = [trace0]
fig = go.Figure(data, layout=layout)
pyplt(fig, filename='plotly/quantile_0916.html')

'plotly/quantile_0916.html'

DPC

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform, RandomNormal
from matplotlib import pyplot as plt

from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold, cross_val_score, train_test_split

tfd = tfp.distributions

In [ ]:
def BNN_normal():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(64, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x = Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  # outputs = Dense(1)(mean_x)
  model = keras.Model(inputs=[mean_inputs], outputs=[outputs])
  return model

def BNN_gamma():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x=Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Gamma(concentration=1e-3+tf.abs(t[..., 0]),rate=1e-3+tf.abs(t[..., 1])))(mean_x)
  model = keras.Model(inputs=mean_inputs,outputs=outputs)
  return model

def BNN_laplace():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x=Dense(2)(mean_x)
  #outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Laplace(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  #outputs = tfp.layers.IndependentNormal(1)(mean_x)
  model = keras.Model(inputs=mean_inputs,outputs=outputs)
  return model

import keras.backend as K
import math as m
def negative_loglikelihood(targets,estimated_distribution):
  return -estimated_distribution.log_prob(targets)

In [ ]:
from scipy.stats import norm
import scipy
import numpy as np
from numba import jit,njit
from tqdm import *

def calculate_kl_divergence(dis1, dis2):
    kl_div = 0
    min_1 = dis1[0] - 2*dis1[1]
    min_2 = dis2[0] - 2*dis2[1]
    max_1 = dis1[0] + 2*dis1[1]
    max_2 = dis2[0] + 2*dis2[1]
    min_value = min(min_1, min_2)
    max_value = max(max_1, max_2)
    values = np.linspace(min_value, max_value, 20)

    for x in values:
        px = norm.pdf(x, loc = dis1[0], scale = dis1[1])
        qx = norm.pdf(x, loc = dis2[0], scale = dis2[1])
        if px < 1e-16:
            px = 1e-16

        if qx < 1e-16:
            qx = 1e-16

        kl_div += px*np.log(px/qx)

    return kl_div

In [ ]:
# For QiPao Plot
distance_matrix = np.zeros((500, 14))


i = 368
nrm0 = nrm[i]
speed0 = nrm0[:, 0::2].flatten()

power_list = []

speed_list = np.linspace(np.percentile(speed0, 0.5), np.percentile(speed0, 99.5), 20)
speed_list = speed_list.reshape(len(speed_list), 1)

for j in range(14):
  model = BNN_gamma()
  model.load_weights("models4plot/density_%s.h5"%j)
  power_list.append(model(speed_list))


tmp_distance = np.zeros((14, 14))
for m in range(14):
  mean_list1 = power_list[m].mean().numpy().tolist()
  std_list1 = power_list[m].stddev().numpy().tolist()
  length = len(mean_list1)
  for j in range(m+1, 14):
    mean_list2 = power_list[j].mean().numpy().tolist()
    std_list2 = power_list[j].stddev().numpy().tolist()
    tmp_distance[m][j] = np.mean([calculate_kl_divergence([mean_list1[k],std_list1[k]], [mean_list2[k],std_list2[k]]) for k in range(length)])
    tmp_distance[j][m] = tmp_distance[m][j]


for k in range(14):
  distance_all = np.delete(tmp_distance[k], k)
  sort_array = np.argsort(distance_all)[:-3]
  filter_distance = [distance_all[i] for i in sort_array]
  distance_matrix[i][k] = np.mean(filter_distance)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)

In [ ]:
# Distance result for quantile power curve
color_array = np.array([ 24.8136569 , 290.59353891,  21.38086794,  19.43169991,
        22.98311105,  47.34687082, 217.06526472,  21.92427844,
        30.86901896,  31.48884859,  34.94920645,  22.18617512,
        15.00589184, 116.35305843])

size_array = color_array/2

pyplt = py.offline.plot

trace0 = go.Scatter(
 x=[10, 20, 30, 40, 50, 10, 20, 30, 40, 50, 10, 20, 30, 40,],
 y=[50, 50, 50, 50, 50, 30, 30, 30, 30, 30, 10, 10, 10, 10,],
 mode='markers',
 marker=dict(
     color=color_array,
     size=size_array,
     showscale= True,
 )
)

layout = go.Layout(
    template = "plotly_white",
)

data = [trace0]
fig = go.Figure(data, layout=layout)
pyplt(fig, filename='plotly/density_0916.html')

'plotly/density_0916.html'

In [ ]:
distance_matrix[i]

array([ 24.8136569 , 290.59353891,  21.38086794,  19.43169991,
        22.98311105,  47.34687082, 217.06526472,  21.92427844,
        30.86901896,  31.48884859,  34.94920645,  22.18617512,
        15.00589184, 116.35305843])

### Control chart monitoring

In [8]:
dis_array1 = np.array([ 24.8136569 , 290.59353891,  21.38086794,  19.43169991,
        22.98311105,  47.34687082, 217.06526472,  21.92427844,
        30.86901896,  31.48884859,  34.94920645,  22.18617512,
        15.00589184, 116.35305843])

dis_array2 = np.array([0.02246418, 0.07281165, 0.02257778, 0.02055283, 0.02263397,
       0.02690489, 0.07162813, 0.02759623, 0.02265561, 0.04322292,
       0.03252698, 0.02271159, 0.02239755, 0.04226735])

Generate data for BCKDE

In [60]:
# Generate simulation data for BCKDE Cumulative plot
df = pd.DataFrame()
df['V1'] = np.linspace(0, np.max(dis_array1), 1000)
df['V2'] = np.linspace(0, np.max(dis_array2), 1000)
df.to_csv('data4BCKDECumulative.csv', index=False)

# Record distance value
df = pd.DataFrame()
df['V1'] = dis_array1
df['V2'] = dis_array2
df.to_csv('distance4BCKDECumulative.csv', index=False)

KDE

In [10]:
import scipy

def calculate_cdf(dis, model, alpha):
  value_list = np.linspace(min(dis)-max(dis), 2*max(dis), 2000)

  for i in range(len(value_list)-1):
      value1 = model.integrate_box_1d(low = -np.Inf, high = value_list[i])
      value2 = model.integrate_box_1d(low = -np.Inf, high = value_list[i+1])
      if value1 <= alpha and value2 >= alpha:
          return (value_list[i] + value_list[i+1])/2

  return np.nan

In [11]:
kde_model1 = scipy.stats.gaussian_kde(dis_array1, bw_method = 0.8)
cdf_list1 = [calculate_cdf(dis_array1, kde_model1, confidence) for confidence in np.linspace(0.01,0.99,99)]

kde_model2 = scipy.stats.gaussian_kde(dis_array2, bw_method = 0.8)
cdf_list2 = [calculate_cdf(dis_array2, kde_model2, confidence) for confidence in np.linspace(0.01,0.99,99)]

In [62]:
df1 = pd.read_csv('cumulativeplot4bckde_bw0.8.csv',)
df_array1 = np.array(df1)

In [ ]:
# BCKDE for DPC bw=0.8
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 10), dpi=300, sharex=False, sharey=False)

ind = np.linspace(0.01,0.99,99).searchsorted(0.95)
marker_list = ['o', '^', 's', 'x', 'D']
color_list = ['#90EE90', 'magenta', 'sandybrown', '#1E90FF', '#f12c60',]
ind = df_array1[:,0].searchsorted(0.9)

alarm_boundary = np.linspace(0, np.max(dis_array1), 1000)[ind]
axes[0].plot(np.linspace(0, np.max(dis_array1), 1000), df_array1[:,0])
axes[0].axvline(alarm_boundary, color='r', label='UCL', linewidth=2,  linestyle='-')
for i in range(14):
  color_id = i//5
  marker_id = i%5
  axes[0].axvline(dis_array1[i], ymin=0.01, ymax=0.99, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.3,  marker=marker_list[marker_id], markersize=5)

axes[0].set_xlabel('Power curve distance',fontsize=13)
axes[0].set_ylabel('Cumulative probability',fontsize=13)
axes[0].set_title('BCKDE for DPC (h=0.8)',fontsize=13)

# BCKDE for QPC bw=0.8
ind = df_array1[:,1].searchsorted(0.9)
alarm_boundary = np.linspace(0, np.max(dis_array2), 1000)[ind]
axes[1].plot(np.linspace(0, np.max(dis_array2), 1000), df_array1[:,1])
axes[1].axvline(alarm_boundary, color='r', linewidth=2,  linestyle='-')
for i in range(14):
  color_id = i//5
  marker_id = i%5
  axes[1].axvline(dis_array2[i], ymin=0.01, ymax=0.99, label='WT #%s'%(i+1), c=color_list[color_id], linewidth=0.3,  marker=marker_list[marker_id], markersize=5)

axes[1].set_xlabel('Power curve distance',fontsize=13)
axes[1].set_ylabel('Cumulative probability',fontsize=13)
axes[1].set_title('BCKDE for QPC (h=0.8)',fontsize=13)
axes[0].legend(bbox_to_anchor=(0, 1.1), loc=3, borderaxespad=0.1, ncol=6, fontsize=13)